# This script is trying to predict the top 28 categories is in GEAR

Imports for libraries and data

In [15]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

input_file = "service_now_sample.csv"
# can do the yes to 1 in read_csv
df = pd.read_csv(input_file, header = 0)

Only need `short_description`, `U_Category_Match`,  `u_category_gear` to get the categories

In [16]:
gear_df = df[['short_description', 'u_category_gear']]
# remove nulls
gear_df = gear_df.replace(np.nan, '', regex=True)

gear_df['eoffer_emod'] = np.where(gear_df['u_category_gear']=='eOffer/eMod - Electronic Offers/Electronic Modifications', 1, 0)
gear_df['vcss'] = np.where(gear_df['u_category_gear']=='VCSS - Vendor Customer Self Service', 1, 0)
gear_df['any_connect_windows'] = np.where(gear_df['u_category_gear']=='Cisco AnyConnect Windows Client 3.1', 1, 0)
gear_df['easi'] = np.where(gear_df['u_category_gear']=='EASi - Electronic Acquisition System Integration', 1, 0)
gear_df['google_email'] = np.where(gear_df['u_category_gear']=='Google Email', 1, 0)
gear_df['pegasys_admin'] = np.where(gear_df['u_category_gear']=='Pegasys Admin Queries', 1, 0)
gear_df['fss_online'] = np.where(gear_df['u_category_gear']=='FSS Online - Federal Supply Service Online', 1, 0)
gear_df['etams'] = np.where(gear_df['u_category_gear']=='ETAMS - Electronic Time and Attendance Management System', 1, 0)
gear_df['pegasys_data'] = np.where(gear_df['u_category_gear']=='PDE - Pegasys Data Entry', 1, 0)
gear_df['aloha'] = np.where(gear_df['u_category_gear']=='Aloha - Authorized Leave and Overtime Help Application', 1, 0)
gear_df['fmis'] = np.where(gear_df['u_category_gear']=='FMIS - Financial Management Information System', 1, 0)
gear_df['apm'] = np.where(gear_df['u_category_gear']=='APM - Acquisition Planning Module', 1, 0)
gear_df['google_docs'] = np.where(gear_df['u_category_gear']=='Google Docs', 1, 0)
gear_df['google_chrome'] = np.where(gear_df['u_category_gear']=='Google Chrome', 1, 0)
gear_df['ears'] = np.where(gear_df['u_category_gear']=='EARS - Enterprise Access Request System', 1, 0)
gear_df['bookit'] = np.where(gear_df['u_category_gear']=='BookIt', 1, 0)
gear_df['rocis'] = np.where(gear_df['u_category_gear']=='ROCIS - RISC/OIRA Consolidated Information System', 1, 0)
gear_df['eviewer'] = np.where(gear_df['u_category_gear']=='eViewer', 1, 0)
gear_df['google_calendar'] = np.where(gear_df['u_category_gear']=='Google Calendar', 1, 0)
gear_df['geco'] = np.where(gear_df['u_category_gear']=='GECO - GSA Enhanced Checkout', 1, 0)
gear_df['ors'] = np.where(gear_df['u_category_gear']=='ORS - Offer Registration System', 1, 0)
gear_df['google_sites'] = np.where(gear_df['u_category_gear']=='Google Sites', 1, 0)
gear_df['bi'] = np.where(gear_df['u_category_gear']=='BI - Business Intelligence Framework', 1, 0)
gear_df['google_hangout'] = np.where(gear_df['u_category_gear']=='Google Hangout', 1, 0)
gear_df['google_groups'] = np.where(gear_df['u_category_gear']=='Google Groups', 1, 0)
gear_df['vitap'] = np.where(gear_df['u_category_gear']=='VITAP - Visual Invoice Tracking and Payment (FoxPro)', 1, 0)
gear_df['ocms'] = np.where(gear_df['u_category_gear']=='OCMS - On-Line Contract Management System', 1, 0)
gear_df['pegasys_vrm'] = np.where(gear_df['u_category_gear']=='Pegasys Vendor Request Management', 1, 0)

In [17]:
# vectorize discription
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(gear_df['short_description'])
X_train_counts.shape

(83933, 21967)

In [18]:
# shape data
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(83933, 21967)

In [19]:
# The most frequently appearing apps in the data
top_apps = ['eoffer_emod', 'vcss', 'any_connect_windows', 'easi', 'google_email', 'pegasys_admin',
           'fss_online', 'etams', 'pegasys_data', 'aloha', 'fmis', 'apm', 'google_docs','google_chrome',
           'ears', 'bookit', 'rocis','eviewer', 'google_calendar','geco', 'ors', 'google_sites', 'bi', 
           'google_hangout', 'google_groups', 'vitap', 'ocms', 'pegasys_vrm']

for app in top_apps:
    formated_category = gear_df[[app]]
    text_clf = MultinomialNB().fit(X_train_counts,formated_category.values.ravel())
    predicted = text_clf.predict(X_train_counts)
    print(app, np.mean(predicted == formated_category.values.ravel()))
    gear_df[app] = predicted


eoffer_emod 0.983784685403834
vcss 0.9886099627083507
any_connect_windows 0.9766003836393313
easi 0.9895273611094564
google_email 0.979173864868407
pegasys_admin 0.9869777084102797
fss_online 0.9906711305446011
etams 0.987859364016537
pegasys_data 0.9936020397221593
aloha 0.9940190389953891
fmis 0.9963780634553752
apm 0.9977482039245589
google_docs 0.9954606650542694
google_chrome 0.9952581225501292
ears 0.9955083221140671
bookit 0.9966640058141613
rocis 0.9979150036338508
eviewer 0.9972954618564808
google_calendar 0.9972597190616325
geco 0.9981771174627382
ors 0.9976528898049635
google_sites 0.9976052327451658
bi 0.9979864892235474
google_hangout 0.9986536880607151
google_groups 0.9986536880607151
vitap 0.9988085735050576
ocms 0.9992374870432369
pegasys_vrm 0.9994519438123265


In [20]:
gear_df.to_csv('category_predictions_top_28.csv')